In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from jive.metrics.novel import flipflopindex_proportion_exceeding

In [2]:
fcst = xr.open_mfdataset(["../data/fcst_2020_2021.nc", "../data/fcst_2021_2022.nc", "../data/fcst_2022_2023.nc"])
fcst = fcst["__xarray_dataarray_variable__"].compute()

In [3]:
def make_cat(da):
    """Convert continuous EHFsev values into categories"""
    da2 = xr.where(fcst>=1, x=1, y=0) # sev or higher
    da3 = xr.where(fcst>=3, x=1, y=0) # extreme
    dacat = da2 + da3
    dacat = dacat.where(~np.isnan(da))
    return dacat

In [4]:
fcst_cat = make_cat(fcst)

In [5]:
fcstsummed = fcst_cat.sum("lead_day")
fcst_heat = fcst_cat.where(fcstsummed > 0)
# Calculate the number of forecast sequences
fcst_sev_seq = fcst_heat > 0
fcst_sev_seq.sum("lead_day").clip(max=1).sum()

<xarray.DataArray '__xarray_dataarray_variable__' ()>
array(2955)

In [6]:
flipflopping = flipflopindex_proportion_exceeding(
    fcst_heat, 
    sampling_dim="lead_day", 
    thresholds = [0.0000001], 
    days012=[0, 1, 2],
    days123=[1, 2, 3],
    days234=[2, 3, 4],
    days345=[3, 4, 5],
    days456=[4, 5, 6],
    days0123=[0, 1, 2, 3],
    days01234=[0, 1, 2, 3, 4],
    days012345=[0, 1, 2, 3, 4, 5],
    days0123456=[0, 1, 2, 3, 4, 5, 6],
)
flipflopping * 100

<xarray.Dataset>
Dimensions:      (threshold: 1)
Coordinates:
  * threshold    (threshold) float64 1e-07
Data variables:
    days012      (threshold) float64 2.301
    days123      (threshold) float64 6.091
    days234      (threshold) float64 4.738
    days345      (threshold) float64 5.99
    days456      (threshold) float64 6.362
    days0123     (threshold) float64 8.494
    days01234    (threshold) float64 16.72
    days012345   (threshold) float64 26.26
    days0123456  (threshold) float64 36.51
Attributes:
    selections:    {'days012': [0, 1, 2], 'days123': [1, 2, 3], 'days234': [2...
    sampling_dim:  lead_day

In [7]:
for i in np.arange(0, 6):

    leadday1 = i
    leadday2 = i + 1
    condition1 = fcst_cat.sel(lead_day=leadday2) > fcst_cat.sel(lead_day=leadday1)
    condition2 = fcst_cat.sel(lead_day=leadday2) < fcst_cat.sel(lead_day=leadday1)
    print(f"lead days {leadday2} - {leadday1}")
    print(f"Decrease count: {condition1.sum().item()}")
    print(f"Increase count: {condition2.sum().item()}")
    print(f"Total: {condition1.sum().item() + condition2.sum().item()}")
    print(f"ratio = {condition1.sum().item() / condition2.sum().item()}")
    print(" ")

lead days 1 - 0
Decrease count: 385
Increase count: 150
Total: 535
ratio = 2.566666666666667
 
lead days 2 - 1
Decrease count: 731
Increase count: 125
Total: 856
ratio = 5.848
 
lead days 3 - 2
Decrease count: 257
Increase count: 260
Total: 517
ratio = 0.9884615384615385
 
lead days 4 - 3
Decrease count: 314
Increase count: 294
Total: 608
ratio = 1.0680272108843538
 
lead days 5 - 4
Decrease count: 311
Increase count: 385
Total: 696
ratio = 0.8077922077922078
 
lead days 6 - 5
Decrease count: 434
Increase count: 434
Total: 868
ratio = 1.0
 


In [ ]:
# Now repeat with just two cateogies - warning vs no warning
for i in np.arange(0, 6):

    leadday1 = i
    leadday2 = i + 1
    warn = fcst_cat.clip(max=1)
    condition1 = warn.sel(lead_day=leadday2) > warn.sel(lead_day=leadday1)
    condition2 = warn.sel(lead_day=leadday2) < warn.sel(lead_day=leadday1)
    print(f"lead days {leadday2} - {leadday1}")
    print(f"Decrease count: {condition1.sum().item()}")
    print(f"Increase count: {condition2.sum().item()}")
    print(f"Total: {condition1.sum().item() + condition2.sum().item()}")
    print(f"ratio = {condition1.sum().item() / condition2.sum().item()}")
    print(" ")